## Estimating Default Probabilities

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
from math import exp
from math import log
from math import sqrt
from functools import partial
import locale

In [2]:
locale.setlocale(locale.LC_ALL, '')

'en_US.UTF-8'

In [3]:
class  CashflowDescriptor:
    '''
    Represents cashflow schedules
    '''
    def  __init__(self, coupon_rate, coupon_frequency, notional, T):
        '''
        :param coupon_rate: coupon rate per annum
        :param coupon_frequency: how many times a year is coupon paid
        :param notional: notional amount due
        :param T: time when the last coupon and notional are due
        '''
        self.coupon_rate = coupon_rate
        self.coupon_frequency = coupon_frequency
        self.notional = notional
        self.T = T
        self.coupon_interval = 1 / coupon_frequency
        self.timeline = np.arange(self.coupon_interval, T+self.coupon_interval, self.coupon_interval)
        self.coupon_amount = notional * coupon_rate * self.coupon_interval
        
    def cashflow(self, t, coupon_rate=None):
        '''
        :param t: future time in years
        :param coupon_rate: a different coupon rate from self.coupon_rate, might be handy when valuing asset swaps
        '''
        coupon_amount = self.coupon_amount if coupon_rate == None else notional * coupon_rate * self.coupon_interval
        if t in self.timeline:  return  coupon_amount + (self.notional if t == self.T else 0)
        else:                   return  0
        
    def pv_cashflows_from_time(self, start_time, discount_rate):
        '''
        Calculates the value of cashflows past 'start_time' as seen at 'start_time'
        :param start_time: a time expressed in years such that all cashflows taking
                           place at that time or after are priced
        :param discount_rate: a discount rate expressed with continuous compounding
        '''
        start = start_time if start_time in self.timeline else self.timeline[self.timeline.searchsorted(start_time)]
        timeline = np.arange(start, self.T+self.coupon_interval, self.coupon_interval)
        return  self.pv_cashflows(timeline, discount_rate, t0=start_time)
        
    def pv_cashflows(self, timeline, discount_rate, t0=0):
        return  sum(map(lambda t: self.cashflow(t) * exp(-discount_rate*(t-t0)), timeline))
    
    def pv_all_cashflows(self, discount_rate, t0=0):
        return  self.pv_cashflows(self.timeline, discount_rate, t0)
    
    # Special method needed to value the floating leg of asset swaps
    def pv_all_cashflows_with_other_coupon_rate(self, other_coupon_rate, discount_rate, t0=0):
        return  sum(map(lambda t: self.cashflow(t, other_coupon_rate) * exp(-discount_rate*(t-t0)), self.timeline))
    
    @staticmethod
    def to_continuous_compounding(rate, comp_freq):
        return rate if comp_freq == 0 else comp_freq * log(1 + rate / comp_freq)

#### Exercise 19.14
Answer from the textbook:
* The implied pobability of default is 2.74% per year

In [4]:
coupon_rate = .04
coupon_frequency = 2  # semiannual payments
risk_free_rate = .03  # continuous compounding
ytm_rate = .05        # continuous compounding
recovery_rate = .3
T = 4
notional = 100

cashflow_descr = CashflowDescriptor(coupon_rate, coupon_frequency, notional, T)

In [5]:
pv_risk_free_bond = cashflow_descr.pv_all_cashflows(risk_free_rate)
pv_corp_bond      = cashflow_descr.pv_all_cashflows(ytm_rate)
print("Present value risk free bond: %s" % locale.currency(pv_risk_free_bond, grouping=True))
print("Present value corporate bond: %s" % locale.currency(pv_corp_bond, grouping=True))

Present value risk free bond: $103.66
Present value corporate bond: $96.19


In [6]:
tl = pd.Series(range(1, cashflow_descr.T+1)) # Defaults can only take place immediately before coupon payment at year's end
table = pd.DataFrame(index=tl, columns=['Recovery Amount', 'Default-Free Value', 'Loss', 'PV of Expected Loss'])
table.index.name = 'Time'

In [7]:
table['Recovery Amount'] = cashflow_descr.notional * recovery_rate
table['Default-Free Value'] = table.index.to_series().apply(lambda x: cashflow_descr.pv_cashflows_from_time(x, risk_free_rate))
table['Loss'] = table['Default-Free Value'] - cashflow_descr.notional * recovery_rate
table['PV of Expected Loss'] = table.apply(lambda row: exp(-risk_free_rate*row.name)*row.Loss, axis=1)

In [8]:
table

,Recovery Amount,Default-Free Value,Loss,PV of Expected Loss
Time,,,,
1,30.0,104.783107,74.783107,72.572932
2,30.0,103.883092,73.883092,69.580476
3,30.0,102.955668,72.955668,66.676460
4,30.0,102.000000,72.000000,63.858271


In [9]:
Q = (pv_risk_free_bond - pv_corp_bond) / table['PV of Expected Loss'].sum()
print("Risk-neutral annual default probability: %.5f%%" % (100*Q))

Risk-neutral annual default probability: 2.73661%


#### Exercise 19.15
Answer from the textbook:
* Q1: 1.57%
* Q2: 2.60%

In [10]:
coupon_rate = .04
coupon_frequency = 1  # annual payments
risk_free_rate = .035 # continuous compounding
ytm1_rate = .045      # continuous compounding
ytm2_rate = .0475     # continuous compounding
recovery_rate = .4
T1 = 3
T2 = 5
notional = 100
default_year_offset = .5 # defaults take place halfway through each year

cashflow_descr1 = CashflowDescriptor(coupon_rate, coupon_frequency, notional, T1)
cashflow_descr2 = CashflowDescriptor(coupon_rate, coupon_frequency, notional, T2)

In [11]:
pv_risk_free_bond1 = cashflow_descr1.pv_all_cashflows(risk_free_rate)
pv_corp_bond1      = cashflow_descr1.pv_all_cashflows(ytm1_rate)
print("Present value risk free %d-year bond: %s" % (cashflow_descr1.T, locale.currency(pv_risk_free_bond1, grouping=True)))
print("Present value %d-year corporate bond: %s" % (cashflow_descr1.T, locale.currency(pv_corp_bond1, grouping=True)))

pv_risk_free_bond2 = cashflow_descr2.pv_all_cashflows(risk_free_rate)
pv_corp_bond2      = cashflow_descr2.pv_all_cashflows(ytm2_rate)
print("Present value risk free %d-year bond: %s" % (cashflow_descr2.T, locale.currency(pv_risk_free_bond2, grouping=True)))
print("Present value %d-year corporate bond: %s" % (cashflow_descr2.T, locale.currency(pv_corp_bond2, grouping=True)))

Present value risk free 3-year bond: $101.23
Present value 3-year corporate bond: $98.35
Present value risk free 5-year bond: $101.97
Present value 5-year corporate bond: $96.24


In [12]:
# Defaults can only take place halfway through each year
tl1 = pd.Series(np.arange(default_year_offset, cashflow_descr1.T, cashflow_descr1.coupon_interval))
table1 = pd.DataFrame(index=tl1, columns=['Recovery Amount', 'Default-Free Value', 'Loss', 'PV of Expected Loss'])
table1.index.name = 'Time'

In [13]:
table1['Recovery Amount'] = cashflow_descr1.notional * recovery_rate
table1['Default-Free Value'] = table1.index.to_series().apply(lambda x: cashflow_descr1.pv_cashflows_from_time(x, risk_free_rate))
table1['Loss'] = table1['Default-Free Value'] - cashflow_descr1.notional * recovery_rate
table1['PV of Expected Loss'] = table1.apply(lambda row: exp(-risk_free_rate*row.name)*row.Loss, axis=1)

In [14]:
table1

,Recovery Amount,Default-Free Value,Loss,PV of Expected Loss
Time,,,,
0.5,40.0,103.012789,63.012789,61.919658
1.5,40.0,102.611458,62.611458,59.409153
2.5,40.0,102.195833,62.195833,56.984995


In [15]:
Q1 = (pv_risk_free_bond1 - pv_corp_bond1) / table1['PV of Expected Loss'].sum()

In [16]:
tl2 = pd.Series(np.arange(default_year_offset, cashflow_descr2.T, cashflow_descr2.coupon_interval)) # Defaults can only take place immediately before coupon payment at year's end
table2 = pd.DataFrame(index=tl2, columns=['Recovery Amount', 'Default-Free Value', 'Loss', 'PV of Expected Loss'])
table2.index.name = 'Time'

In [17]:
table2['Recovery Amount'] = cashflow_descr2.notional * recovery_rate
table2['Default-Free Value'] = table2.index.to_series().apply(lambda x: cashflow_descr2.pv_cashflows_from_time(x, risk_free_rate))
table2['Loss'] = table2['Default-Free Value'] - cashflow_descr2.notional * recovery_rate
table2['PV of Expected Loss'] = table2.apply(lambda row: exp(-risk_free_rate*row.name)*row.Loss, axis=1)

In [18]:
table2

,Recovery Amount,Default-Free Value,Loss,PV of Expected Loss
Time,,,,
0.5,40.0,103.774514,63.774514,62.668169
1.5,40.0,103.400316,63.400316,60.157664
2.5,40.0,103.012789,63.012789,57.733506
3.5,40.0,102.611458,62.611458,55.392727
4.5,40.0,102.195833,62.195833,53.132458


In [19]:
PV1 = table2['PV of Expected Loss'].iloc[:cashflow_descr1.T].sum()
PV2 = table2['PV of Expected Loss'].iloc[cashflow_descr1.T:].sum()
print(PV1, PV2)

180.55933866522133 108.52518451951184


In [20]:
Q2 = (pv_risk_free_bond2 - pv_corp_bond2 - Q1*PV1) / PV2

In [21]:
print("Q1 (Risk-neutral annual default probability for first %d years): %.5f%%" % (cashflow_descr1.T, 100*Q1))
print("Q2 (Risk-neutral annual default probability for last %d years): %.5f%%"
      % (cashflow_descr2.T-cashflow_descr1.T, 100*Q2))

Q1 (Risk-neutral annual default probability for first 3 years): 1.61489%
Q2 (Risk-neutral annual default probability for last 2 years): 2.59462%


#### Exercise 19.18
Answer from the textbook:
* V<sub>0</sub>=$6.80M
* &sigma;<sub>V</sub>=14.82%
* Probability of default=1.15%

In [22]:
E0 = 2_000_000 # Current value of company's equity
sigma_E = .5   # Annual volatility of company's equity
T = 1          # Debt is due in 1 year
D = 5_000_000  # Debt due in T years
risk_free_rate = .04  # continuous compounding

To solve two non-linear equations of the form F(x, y) = 0 and G(x, y) = 0, I use SciPy's `optimize` module and search for an (x, y) tuple that minimizes $\left[F(x, y)\right]^2 + \left[G(x, y)\right]^2$. In the below cell `obj_func_1` corresponds to the equation $V_0\cdot N(d_1) - D\cdot e^{-r\cdot T}\cdot N(d_2) - E_0 = 0$. While `obj_func_2` corresponds to the following equation derived through Ito's lemma:
$N(d_1)\cdot \sigma_V\cdot V_0 - \sigma_E\cdot E_0 = 0$.

In [23]:
# Define d1, d2, and obj_func_1 as per the Black-Scholes-Merton formula
def  d1(V0, sigma, D, r, T):
    return  (log(V0/D) + (r + sigma**2/2) * T) / (sigma * sqrt(T))
def  d2(V0, sigma, D, r, T):
    return  d1(V0, sigma, D, r, T) - sigma * sqrt(T)
def  obj_func_1(V0, sigma, E0, D, r, T):
    return  V0 * norm.cdf(d1(V0, sigma, D, r, T)) - D * exp(-r*T)*norm.cdf(d2(V0, sigma, D, r, T)) - E0
def  obj_func_2(V0, sigma, sigma_equity, E0, D, r, T):
    return  norm.cdf(d1(V0, sigma, D, r, T)) * sigma * V0 - sigma_equity * E0

# We'll minimize obj_func_1^2 + obj_func_2^2
def  obj_func(V0, sigma, sigma_equity, E0, D, r, T):
    return  obj_func_1(V0, sigma, E0, D, r, T)**2 + obj_func_2(V0, sigma, sigma_equity, E0, D, r, T)**2

In [24]:
# Now let's bind the arguments whose values we know, this gives us a function of two arguments -- V0 and sigma
objective_function_aux = partial(obj_func, sigma_equity=sigma_E, E0=E0, D=D, r=risk_free_rate, T=T)
objective_function = lambda x: objective_function_aux(x[0], x[1])

In [25]:
x0 = (1_000_000, 0.3)        # starting with assuming V0 = $1M and volatility of assets 30%
res = minimize(objective_function, x0, method = 'Nelder-Mead')
if res.success:
    print('V0=%s, \u03C3v=%.2f%%' % (locale.currency(res.x[0], grouping=True), res.x[1] * 100))
    print('Probability of default in %d year(s): %.2f%%' % (T, 100*norm.cdf(-d2(res.x[0], res.x[1], D, risk_free_rate, T))))

V0=$6,801,247.19, σv=14.82%
Probability of default in 1 year(s): 1.15%


#### Exercise 19.22
Answer from the textbook:
* Price risk-less bond: \$95.3579
* PV of expected loss from default: \$5.3579
* PV of fixed leg: \$105.3579
* Asset-swap spread: 1.2592%

In [26]:
coupon_rate = .05
coupon_frequency = 2   # semiannual payments
libor_swap_rate = .06  # continuous compounding
T = 5                  # it's a five-year bond
notional = 100
pv_bond = 90.

cashflow_descr = CashflowDescriptor(coupon_rate, coupon_frequency, notional, T)

In [27]:
pv_riskfree_bond = cashflow_descr.pv_all_cashflows(libor_swap_rate)
pv_fixed_leg     = notional - pv_bond + pv_riskfree_bond

In [28]:
print('Price risk-less bond: $%.4f' % pv_riskfree_bond)
print('PV of expected loss from default: $%.4f' % (pv_riskfree_bond-pv_bond))
print('PV of fixed leg: $%.4f' % pv_fixed_leg)

Price risk-less bond: $95.3579
PV of expected loss from default: $5.3579
PV of fixed leg: $105.3579


The reason the PV of the fixed leg is higher than the price of the risk-less bond by $10 is because the payer of the swap needs to pay the buyer the initial difference between the notional and the PV of the bond.

In [29]:
# Now we need to find a spread=libor_swap_rate+asset_swap_rate paying which will result in an equal PV over the same notional.
# This is equivallent to minimizing this function
objective_function = lambda spread: abs(cashflow_descr.pv_all_cashflows_with_other_coupon_rate(spread, libor_swap_rate)-pv_fixed_leg)
spread = minimize_scalar(objective_function, bounds=(0, 10), method='bounded') # looking for a spread in [0%-1000%]

# We need to convert the libor swap rate from continuous compounding to half-yearly compounding when calculating
# the asset swap spread
libor_swap_rate_hy = 2 * (sqrt(exp(libor_swap_rate)) - 1)
if spread.success:
    print('Asset swap spread: %.4f%%' % ((spread.x-libor_swap_rate_hy) * 100))

Asset swap spread: 1.2592%


In [30]:
# Verify that the asset swap spread is calculated correctly
cashflow_descr2 = CashflowDescriptor(spread.x, coupon_frequency, notional, T)
assert abs(pv_fixed_leg - cashflow_descr2.pv_all_cashflows(libor_swap_rate)) < 1e-3, 'Asset swap spread isn\'t calculated correctly'

Now let's calculate the asset swap spread using the result from exercise 19.16 &mdash; namely that the present value of the spread must equal `pv_riskfree_bond - pv_bond`.

In [31]:
# First let's figure out the present value of $1 received at the same frequency as coupons. This is
# the present value of a spread of 2% per annum that pays coupons half-yearly.
# This is equivallent to pricing an annuity that pays $1 every 6 months.

# This is the present value of a riskless bond with a notional of $100 that pays 2% coupon every 6 months minus
# the present value of the bond notional.
cashflow_descr3 = CashflowDescriptor(0.02, coupon_frequency, notional, T)
one_dollar_annuity = cashflow_descr3.pv_all_cashflows(libor_swap_rate) - notional*exp(-libor_swap_rate*(T))

# Or we can price it using the standard formula for annuity.
# For which we'll need to switch to half-yearly compounding frequency.
r = libor_swap_rate_hy / 2
one_dollar_annuity_ = (1. - (1 + r)**-len(cashflow_descr3.timeline)) / r

assert abs(one_dollar_annuity - one_dollar_annuity_) < 1e-10, '$1 annuity isn\'t calculated correctly'
print('Present value of $1 annuity: {:.4f}'. format(one_dollar_annuity))

Present value of $1 annuity: 8.5104


In [32]:
# Now let's see by how much the half-yearly annuity payment must be adjusted to equal the present value of
# pv_riskfree_bond - pv_bond.

# Since the present value of a spread of 2% is calculated as a sum of half-yearly spread payments discounted
# with riskless rate, the factor by which the present value of the required spread exceeds the 2% spread
# is the factor that when applied to the 2% spread gives us the required asset spread.
required_annuity_payment = (pv_riskfree_bond - pv_bond) / one_dollar_annuity

# The asset swap spread is therefore `required_annuity_payment` expressed as an annual rate with semiannual
# compounding frequency.
print('Asset swap spread: {:.4f}%'.format(required_annuity_payment*2))

Asset swap spread: 1.2591%


#### Exercise 19.23

In [33]:
coupon_rate = .07
coupon_frequency = 2  # semiannual payments
ytm_rate = .05        # semiannual compounding
ytm_rf_rate = .04     # semiannual compounding
risk_free_rate = CashflowDescriptor.to_continuous_compounding(ytm_rf_rate, 2) # continuous compounding
recovery_rate = .45
T = 3
notional = 100
default_year_offset = .5 # defaults take place halfway through each year

cashflow_descr = CashflowDescriptor(coupon_rate, coupon_frequency, notional, T)

In [34]:
pv_risk_free_bond = cashflow_descr.pv_all_cashflows(risk_free_rate)
pv_corp_bond      = cashflow_descr.pv_all_cashflows(CashflowDescriptor.to_continuous_compounding(ytm_rate, 2))
print("Present value risk free bond: %s" % locale.currency(pv_risk_free_bond, grouping=True))
print("Present value corporate bond: %s" % locale.currency(pv_corp_bond, grouping=True))

Present value risk free bond: $108.40
Present value corporate bond: $105.51


In [35]:
# Defaults can only take place immediately before coupon payment every 6 months
tl = pd.Series(np.arange(default_year_offset, cashflow_descr.T + cashflow_descr.coupon_interval,
                         cashflow_descr.coupon_interval))
table = pd.DataFrame(index=tl, columns=['Recovery Amount', 'Default-Free Value', 'Loss', 'PV of Expected Loss'])
table.index.name = 'Time'

In [36]:
table['Recovery Amount'] = cashflow_descr.notional * recovery_rate
table['Default-Free Value'] = table.index.to_series().apply(lambda x: cashflow_descr.pv_cashflows_from_time(x, risk_free_rate))
table['Loss'] = table['Default-Free Value'] - cashflow_descr.notional * recovery_rate
table['PV of Expected Loss'] = table.apply(lambda row: exp(-risk_free_rate*row.name)*row.Loss, axis=1)

In [37]:
table

,Recovery Amount,Default-Free Value,Loss,PV of Expected Loss
Time,,,,
0.5,45.0,110.570189,65.570189,64.284499
1.0,45.0,109.211593,64.211593,61.718179
1.5,45.0,107.825825,62.825825,59.202178
2.0,45.0,106.412341,61.412341,56.735511
2.5,45.0,104.970588,59.970588,54.317209
3.0,45.0,103.500000,58.500000,51.946326


In [38]:
pv_loss_from_default = pv_risk_free_bond - pv_corp_bond
uncond_prob = pv_loss_from_default / table['PV of Expected Loss'].sum()

In [39]:
print('Unconditional semiannual default probability: {:.5f}'.format(uncond_prob))

Unconditional semiannual default probability: 0.00831


Calculating conditional default probabilities is a little trickier and requires solving a small optimization problem.

In [40]:
def objective_function(q_cond):
    series = table['PV of Expected Loss']
    sum = 0.
    for i, idx in enumerate(series.index):
        sum += series[idx] * q_cond * (1 - q_cond)**i
    return abs(sum - pv_loss_from_default)

# looking for a probability implies the [0, 1] range
cond_prob = minimize_scalar(objective_function, bounds=(0, 1), method='bounded')
if cond_prob.success:
    print('Conditional semiannual default probability: {:.5f}'.format(cond_prob.x))

Conditional semiannual default probability: 0.00848


#### Exercise 19.24

In [41]:
coupon_rate = .08
coupon_frequency = 1  # annual payments
risk_free_rate = .045 # continuous compounding
ytm1_rate = .06       # continuous compounding
ytm2_rate = .066      # continuous compounding
recovery_rate = .35
T1 = 1
T2 = 2
notional = 100
default_year_offset = .5 # defaults take place halfway through each year

cashflow_descr1 = CashflowDescriptor(coupon_rate, coupon_frequency, notional, T1)
cashflow_descr2 = CashflowDescriptor(coupon_rate, coupon_frequency, notional, T2)

In [42]:
pv_risk_free_bond1 = cashflow_descr1.pv_all_cashflows(risk_free_rate)
pv_corp_bond1      = cashflow_descr1.pv_all_cashflows(ytm1_rate)
print("Present value risk free %d-year bond: %s" % (cashflow_descr1.T, locale.currency(pv_risk_free_bond1, grouping=True)))
print("Present value %d-year corporate bond: %s" % (cashflow_descr1.T, locale.currency(pv_corp_bond1, grouping=True)))

pv_risk_free_bond2 = cashflow_descr2.pv_all_cashflows(risk_free_rate)
pv_corp_bond2      = cashflow_descr2.pv_all_cashflows(ytm2_rate)
print("Present value risk free %d-year bond: %s" % (cashflow_descr2.T, locale.currency(pv_risk_free_bond2, grouping=True)))
print("Present value %d-year corporate bond: %s" % (cashflow_descr2.T, locale.currency(pv_corp_bond2, grouping=True)))

Present value risk free 1-year bond: $103.25
Present value 1-year corporate bond: $101.71
Present value risk free 2-year bond: $106.35
Present value 2-year corporate bond: $102.13


In [43]:
# Defaults can only take place halfway through each year
tl1 = pd.Series(np.arange(default_year_offset, cashflow_descr1.T, cashflow_descr1.coupon_interval))
table1 = pd.DataFrame(index=tl1, columns=['Recovery Amount', 'Default-Free Value', 'Loss', 'PV of Expected Loss'])
table1.index.name = 'Time'

In [44]:
table1['Recovery Amount'] = cashflow_descr1.notional * recovery_rate
table1['Default-Free Value'] = table1.index.to_series().apply(lambda x: cashflow_descr1.pv_cashflows_from_time(x, risk_free_rate))
table1['Loss'] = table1['Default-Free Value'] - cashflow_descr1.notional * recovery_rate
table1['PV of Expected Loss'] = table1.apply(lambda row: exp(-risk_free_rate*row.name)*row.Loss, axis=1)

In [45]:
table1

,Recovery Amount,Default-Free Value,Loss,PV of Expected Loss
Time,,,,
0.5,35.0,105.597134,70.597134,69.026435


In [46]:
Q1 = (pv_risk_free_bond1 - pv_corp_bond1) / table1['PV of Expected Loss'].sum()

In [47]:
tl2 = pd.Series(np.arange(default_year_offset, cashflow_descr2.T, cashflow_descr2.coupon_interval)) # Defaults can only take place immediately before coupon payment at year's end
table2 = pd.DataFrame(index=tl2, columns=['Recovery Amount', 'Default-Free Value', 'Loss', 'PV of Expected Loss'])
table2.index.name = 'Time'

In [48]:
table2['Recovery Amount'] = cashflow_descr2.notional * recovery_rate
table2['Default-Free Value'] = table2.index.to_series().apply(lambda x: cashflow_descr2.pv_cashflows_from_time(x, risk_free_rate))
table2['Loss'] = table2['Default-Free Value'] - cashflow_descr2.notional * recovery_rate
table2['PV of Expected Loss'] = table2.apply(lambda row: exp(-risk_free_rate*row.name)*row.Loss, axis=1)

In [49]:
table2

,Recovery Amount,Default-Free Value,Loss,PV of Expected Loss
Time,,,,
0.5,35.0,108.772604,73.772604,72.131255
1.5,35.0,105.597134,70.597134,65.989098


In [50]:
PV1 = table2['PV of Expected Loss'].iloc[:cashflow_descr1.T].sum()
PV2 = table2['PV of Expected Loss'].iloc[cashflow_descr1.T:].sum()
print(PV1, PV2)

72.13125456219066 65.98909778773168


In [51]:
Q2 = (pv_risk_free_bond2 - pv_corp_bond2 - Q1*PV1) / PV2

In [52]:
print('Q1 (Risk-neutral annual default probability for first {:d} years): {:.5%}'.format(
    cashflow_descr1.T, Q1))
print('Q2 (Risk-neutral annual default probability for last {:d} years): {:.5%}'.format(
    cashflow_descr2.T-cashflow_descr1.T, Q2))

Q1 (Risk-neutral annual default probability for first 1 years): 2.22691%
Q2 (Risk-neutral annual default probability for last 1 years): 3.95880%


#### Exercise 19.25
The exercise is similar to Ex 19.18. I will reuse the objective functions defined for it.

In [53]:
E0 = 4_000_000 # Current value of company's equity
sigma_E = .6   # Annual volatility of company's equity
T = 2          # Debt is due in 2 years
D = 15_000_000  # Debt due in T years
risk_free_rate = .06  # continuous compounding

In [54]:
# Let's bind the arguments whose values we know, this gives us a function of two arguments -- V0 and sigma
objective_function_aux = partial(obj_func, sigma_equity=sigma_E, E0=E0, D=D, r=risk_free_rate, T=T)
objective_function = lambda x: objective_function_aux(x[0], x[1])

In [55]:
x0 = (1_000_000, 0.3)        # starting with assuming V0 = $1M and volatility of assets 30%
res = minimize(objective_function, x0, method = 'Nelder-Mead')

The market value of the debt is V<sub>0</sub> - E<sub>0</sub>.

In [56]:
if res.success:
    print('V0={:s}, \u03C3v={:.2%}'.format(locale.currency(res.x[0], grouping=True), res.x[1]))
    print('Probability of default in {:d} year(s): {:.2%}'.format(
          T, norm.cdf(-d2(res.x[0], res.x[1], D, risk_free_rate, T))) )
    debt_mkt_val = res.x[0] - E0
    print('Current market value of debt: {:s}'.format(locale.currency(debt_mkt_val, grouping=True)))
    pv_debt_payment = D * exp(-risk_free_rate * T)
    pv_expected_loss = pv_debt_payment - debt_mkt_val
    print('Present value of expected loss from default: {:s}'.format(locale.currency(pv_expected_loss, grouping=True)))
    print('Estimate of recovery rate: {:.2%}'.format(1 - pv_expected_loss/pv_debt_payment))

V0=$17,083,946.65, σv=15.76%
Probability of default in 2 year(s): 15.61%
Current market value of debt: $13,083,946.65
Present value of expected loss from default: $219,859.90
Estimate of recovery rate: 98.35%
